# tune vl的模型，全能的

In [1]:
import numpy as np
import pandas as pd
base_dir = '/data/coding/patent_qa/train/'
df_question = pd.read_json("/data/coding/patent_qa/train/questions.jsonl",lines=True)
quesion_vector = np.load('all_train_question_vectors.npy')
train_ocr_page_num_mapping = pd.read_csv('train_ocr_page_num_mapping.csv')
train_ocr_vectors = np.load('train_ocr_vectors.npy')
train_pdf_image_vectors = np.load("train_pdf_img_vectors.npy") # 前400个
train_pdf_image_page_num_mapping = pd.read_csv('train_pdf_img_page_num_mapping.csv') # 前400个

In [2]:
# 给定了图片的情况下，有图片本身，也需要召回对应的文本
def get_similar_text_embedding(base_dir,document_name,question_idx,top_k):
    document_name = df_question.document[question_idx].split('.')[0]
    vec_idx = train_ocr_page_num_mapping[train_ocr_page_num_mapping['file_name']==document_name]['index'].values
    candidate_vec = train_ocr_vectors[vec_idx]
    query_vec = quesion_vector[question_idx]
    cos_sim = np.dot(candidate_vec, query_vec) / (np.linalg.norm(candidate_vec) * np.linalg.norm(query_vec))
    # 获取最相似的top_k个索引
    top_k_indices = np.argsort(cos_sim)[-top_k:][::-1]
    retrived_idx = vec_idx[top_k_indices] # 最相近的top_k个
    retrived_page_num = train_ocr_page_num_mapping.loc[retrived_idx]['page_num'].to_list()
    # text_list = []
    # for i in range(len(retrived_page_num)):
    #     text_file = base_dir + '/pdf_ocr/' + document_name + '/' + str(retrived_page_num[i]) +'.txt'
    #     with open(text_file,'r',encoding='utf-8') as f:
    #         text_list.append(f.read())
    return retrived_page_num # 返回一个list,大小最大为top_k


def get_similar_image_embedding(base_dir,document_name,question_idx,top_k):
    document_name = df_question.document[question_idx].split('.')[0]
    vec_idx = train_pdf_image_page_num_mapping[train_pdf_image_page_num_mapping['file_name']==document_name]['index'].values
    candidate_vec = train_pdf_image_vectors[vec_idx]
    query_vec = quesion_vector[question_idx]
    cos_sim = np.dot(candidate_vec, query_vec) / (np.linalg.norm(candidate_vec) * np.linalg.norm(query_vec))
    # 获取最相似的top_k个索引
    top_k_indices = np.argsort(cos_sim)[-top_k:][::-1]
    retrived_idx = vec_idx[top_k_indices] # 最相近的top_k个
    retrived_page_num = train_pdf_image_page_num_mapping.loc[retrived_idx]['page_num'].to_list()
    # image_list = []
    # for i in range(len(retrived_page_num)):
    #     image_file = base_dir + '/pdf_img/' + document_name + '/' + str(retrived_page_num[i]) +'.jpg'
    #     image_list.append(image_file)
    return retrived_page_num # 返回一个list,大小最大为top_k

def get_text_answer_vl(document_name,question,options,quesion_idx):
    prompt ="你是一个专利内容分析专家，请根据我提供的专利内容回答我的问题。\n"
    prompt += "【我的问题】【"
    prompt += (question +"】\n")
    prompt += "【选项】【"
    prompt += (' '.join(options) + "】\n")
    prompt += ("专利内容为：\n")
    retrived_page_list = get_similar_text_embedding(base_dir,document_name,quesion_idx,2)
    # 排序
    retrived_page_num = sorted(retrived_page_list)
    retrived_list = []
    for i in range(len(retrived_page_num)):
        text_file = base_dir + '/pdf_ocr/' + document_name.split('.')[0] + '/' + str(retrived_page_num[i]) +'.txt'
        with open(text_file,'r',encoding='utf-8') as f:
            retrived_list.append(f.read())
    prompt += '\n'.join(retrived_list)
    prompt += ("\n\n请你分析专利内容后，回答我的单选题，回答正确选项字母，你的答案为：\n")
    #sampling_params = SamplingParams(temperature=0)
    #outputs = text_qa_llm.generate(question,sampling_params)
    #return outputs[0].outputs[0].text
    messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                ],
            }
        ]
    query = prompt
    # data = dict()
    # data['messages'] = messages
    # infer_requests = [InferRequest(**data)]
    # response = infer_batch(engine, infer_requests)
    return query,[]

def get_image_answer(document_name,question,options,question_idx):
    question1 ="你是一个专利内容分析专家，请根据我提供的专利内容回答我的单选题。\n"
    question1 += "【我的问题】【"
    question1 += (question +"】\n")
    question1 += "【选项】【"
    question1 += (' '.join(options) + "】\n")
    question1 += ("专利内容为：\n")
    retrived_page_list = get_similar_image_embedding(base_dir,document_name,question_idx,2)
    # 排序
    retrived_page_num = sorted(retrived_page_list)
    retrived_list = []
    for i in range(len(retrived_page_num)):
        image_file = base_dir + '/pdf_img/' + document_name.split('.')[0] + '/' + str(retrived_page_num[i]) +'.jpg'
        retrived_list.append(image_file)
    question2 = ("\n\n请你分析专利内容后，回答我的单选题，直接回答正确选项字母，你的答案为：\n")
    if len(retrived_list)>1:
        query = ''
        query += question1
        query += '<image>'
        query += '<image>'
        query += question2
        images = []
        images.append(retrived_list[0])
        images.append(retrived_list[1])
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": retrived_list[0],
                    },
                    {
                        "type": "image",
                        "image": retrived_list[1],
                    },
                    {"type": "text", "text": question2},
                ],
            }
        ]
    else:
        query = ''
        query += question1
        query += '<image>'
        query += question2
        images = []
        images.append(retrived_list[0])
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": retrived_list[0],
                    },
                    {"type": "text", "text": question2},
                ],
            }
        ]
    # data = dict()
    # data['messages'] = messages
    # infer_requests = [InferRequest(**data)]
    # response = infer_batch(engine, infer_requests)
    return query,images

def get_mix_answer(document_name,pic_page_num,question,options,question_idx):
    question1 ="你是一个专利内容分析专家，请根据我提供的专利内容回答我的问题。\n"
    question1 += "【我的问题】【"
    question1 += (question +"】\n")
    question1 += "【选项】【"
    question1 += (' '.join(options) + "】\n")
    question1 += ("该问题直接指向的专利页内容为：\n")
    retrived_page_list = get_similar_text_embedding(base_dir,document_name,question_idx,2)
    # 排序
    retrived_page_num = sorted(retrived_page_list)
    retrived_list = []
    for i in range(len(retrived_page_num)):
        text_file = base_dir + '/pdf_ocr/' + document_name.split('.')[0] + '/' + str(retrived_page_num[i]) +'.txt'
        with open(text_file,'r',encoding='utf-8') as f:
            retrived_list.append(f.read())
    question2 = ("\n\n除了问题直接指向的专利页外，该专利其他相关内容为：\n")
    question3 = ("\n\n请你分析专利内容后，回答我的单选题，直接回答正确选项字母，你的答案为：\n")
    messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg',
                    },
                    {"type": "text", "text": question2},
                    {"type": "text", "text": '\n'.join(retrived_list)},
                    {"type": "text", "text": question3},
                ],
            }
    ]
    query = ''
    query += question1
    query += '<image>'
    query += '\n'.join(retrived_list) 
    query += question2
    images = [base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg']
    # data = dict()
    # data['messages'] = messages
    # infer_requests = [InferRequest(**data)]
    # response = infer_batch(engine, infer_requests)
    return query, images


def get_mix_answer_img(document_name,pic_page_num,question,options,question_idx):
    question1 ="你是一个专利内容分析专家，请根据我提供的专利内容回答我的问题。\n"
    question1 += "【我的问题】【"
    question1 += (question +"】\n")
    question1 += "【选项】【"
    question1 += (' '.join(options) + "】\n")
    question1 += ("该问题直接指向的专利页内容为：\n")
    #retrived_list = get_similar_text_embedding(base_dir,document_name,question_idx,2)
    retrived_page_list = get_similar_image_embedding(base_dir,document_name,question_idx,2)
    # 排序
    retrived_page_num = sorted(retrived_page_list)
    retrived_list = []
    for i in range(len(retrived_page_num)):
        image_file = base_dir + '/pdf_img/' + document_name.split('.')[0] + '/' + str(retrived_page_num[i]) +'.jpg'
        retrived_list.append(image_file)
    question2 = ("\n\n除了问题直接指向的专利页外，该专利其他相关内容为：\n")
    question3 = ("\n\n请你分析专利内容后，回答我的单选题，直接回答正确选项字母，你的答案为：\n")
    if len(retrived_list)>1:
        query = ''
        query += question1
        query += '<image>'
        query += '<image>'
        query += '<image>'
        query += question2
        images = []
        images.append(base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg')
        images.append(retrived_list[0])
        images.append(retrived_list[1])
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg',
                    },
                    {"type": "text", "text": question2},
                    {
                        "type": "image",
                        "image": retrived_list[0],
                    },
                    {
                        "type": "image",
                        "image": retrived_list[1],
                    },
                    {"type": "text", "text": question3},
                ],
            }
        ]
    else:
        query = ''
        query += question1
        query += '<image>'
        query += question2
        images = []
        images.append(base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg')
        images.append(retrived_list[0])
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg',
                    },
                    {
                        "type": "image",
                        "image": retrived_list[0],
                    },
                    {"type": "text", "text": question2},
                ],
            }
        ]

    return query, images



In [3]:
def write_to_jsonl_text_train_set(x):
    tmp_dict = dict()
    tmp_dict['query'] = x['query']
    tmp_dict['document'] = x['document']
    tmp_dict['question_idx'] = int(x['question_idx'])
    tmp_dict['response'] = x['response']
    with open('train_dataset_for_text_0609.jsonl', 'a', encoding='utf-8') as f:
        f.write(json.dumps(tmp_dict, ensure_ascii=False) + '\n')

def write_to_jsonl_img_train_set(x):
    tmp_dict = dict()
    tmp_dict['query'] = x['query']
    tmp_dict['document'] = x['document']
    tmp_dict['question_idx'] = int(x['question_idx'])
    tmp_dict['response'] = x['response']
    tmp_dict['images'] = x['images']
    with open('train_dataset_for_image_0613.jsonl', 'a', encoding='utf-8') as f:
        f.write(json.dumps(tmp_dict, ensure_ascii=False) + '\n')


import re
from tqdm import trange
import json
for i in trange(len(df_question)):
    question = df_question.loc[i,'question']
    document_name = df_question.loc[i,'document']
    options = df_question.loc[i,'options']
    true_answer = df_question.loc[i,'answer']
    full_question = question + ' '.join(options)
    answer = ''
    text_answer = ''
    image_answer = ''
    if "第" in question and "页" in question and "图": # 问题含有图片
        pic_page_num = re.findall(r"第(\d+)页", question)[0]
        pic_page_num = int(pic_page_num)
        query,images = get_mix_answer(document_name,pic_page_num,question,options,i)
        result_dict = dict()
        result_dict['question_idx'] = i
        result_dict['document'] = document_name
        result_dict['query'] = query
        result_dict['response'] = true_answer
        result_dict['images'] = images
        #write_to_jsonl_img_train_set(result_dict)
        query,images = get_mix_answer_img(document_name,pic_page_num,question,options,i)
        result_dict = dict()
        result_dict['question_idx'] = i
        result_dict['document'] = document_name
        result_dict['query'] = query
        result_dict['response'] = true_answer
        result_dict['images'] = images
        write_to_jsonl_img_train_set(result_dict)
    else:
        query,images = get_text_answer_vl(document_name,question,options,i) # 使用文本来回答
        result_dict = dict()
        result_dict['question_idx'] = i
        result_dict['document'] = document_name
        result_dict['query'] = query
        result_dict['response'] = true_answer
        write_to_jsonl_text_train_set(result_dict)
        query,images = get_image_answer(document_name,question,options,i) # 使用图像来回答
        result_dict = dict()
        result_dict['question_idx'] = i
        result_dict['document'] = document_name
        result_dict['query'] = query
        result_dict['response'] = true_answer
        result_dict['images'] = images
        write_to_jsonl_img_train_set(result_dict)
    #break



100%|██████████| 300/300 [00:00<00:00, 423.90it/s]


In [4]:
print(document_name)
print(question)
print(options)

CN110758510A.pdf
观察文件中第6页的图片，编号为12的部件是什么？
['A. 把手', 'B. 刹车', 'C. 旋转挡板', 'D. 锁紧扳手']


In [5]:
options

['A. 把手', 'B. 刹车', 'C. 旋转挡板', 'D. 锁紧扳手']

In [6]:
query,images = get_text_answer_vl(document_name,question,options,i)
print(query)

你是一个专利内容分析专家，请根据我提供的专利内容回答我的问题。
【我的问题】【观察文件中第6页的图片，编号为12的部件是什么？】
【选项】【A. 把手 B. 刹车 C. 旋转挡板 D. 锁紧扳手】
专利内容为：
附图2为本发明所述的PCB治具周转车的放置板的放大立体图；
其中：1、车架；2、第一导轨；3、第二导轨；4、第三导轨；5、第四导轨；6、第一放置架；7、第二放置架；8、第三放置架；9、第四放置架；10、放置板；11、把手；12、刹车；13、旋转挡板；14、锁紧扳手；15、滑块。

具体实施方式
[0011]下面结合附图及具体实施例对本发明作进一步的详细说明。
[0012]请参阅图1-2，本发明所述的PCB治具周转车，包括：车架1；所述车架1呈长方体框架结构；所述车架1底部设有行走轮(未标出)，所述行走轮带有刹车12；所述车架1的一侧设有把手11，所述把手11用来推动所述PCB周转车移动；所述车架1的顶端的长度方向上设有水平设置的第一导轨2和第二导轨3，且所述第一导轨2与第二导轨3平行；所述车架1的下部的长度方向上水平设置有第三导轨4和第四导轨5，所述第三导轨4与第四导轨5平行；所述第三导轨4与第一导轨2处于同一竖直面上，所述第四导轨5与第二导轨3处于同一竖直面上；所述第一导轨2、第二导轨3、第三导轨4和第四导轨5上分别设有四个滑块15，每四个滑块15为一组，每组滑块处于一个竖直面上，且每组滑块所处的竖直面垂直于所述第一导轨2、第二导轨3、第三导轨4以及第四导轨5之间；四组滑块上分别设有第一放置架7、第二放置架8、第三放置架9以及第四放置架10；所述第一放置架7、第二放置架8、第三放置架9以及第四放置架10分别竖直设置，且相互平行；所述第一放置架7、第二放置架8、第三放置架9以及第四放置架10上分别设有水平、间隔且相互平行的放置板10；所述放置板10呈“L”形，所述放置板10的内端设有封闭挡板(未标出)，外端为开口部；所述放置板10的放置表面上设有若干个波浪形凸起(未标出)，所述波浪形凸起能方便PCB治具推入；所述第一放置架6上的放置板10与所述第二放置架7上的放置板10一一对应设置，且每两个对应的所述放置板10相互配合地放置PCB治具；同样地，所述第三放置架8上的放置板10与所述第四放置架9上的放置板10一一对应设置，且每两个对应的所述放置板10相互配合地放